In [2]:
!pip install keras==2.2.4

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 312 kB 722 kB/s eta 0:00:01


In [3]:
!pip install gensim

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 24.2 MB 145.2 MB/s eta 0:00:01   |██████████████████▋             | 14.1 MB 1.2 MB/s eta 0:00:09
     |████████████████████████████████| 103 kB 69.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 103.4 MB/s eta 0:00:01
     |████████████████████████████████| 118 kB 77.9 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 73.9 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 68.2 MB/s eta 0:00:01
     |████████████████████████████████| 6.3 MB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 547 kB 3.5 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-2.0.0-py3-none-any.whl size=101341 sha256=6dd5a63353108fd0bca1835387afc8b98e1c3f9577093ab3bba78e403b5a7e74
  Stored in directory: /home/tione/.cache/pip/wheels/54/7f/ad/18624c030ede84ebb5f4bcd3a6cdfab7

In [4]:
!pip install h5py

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple


In [1]:
from keras.models import Model
from keras.optimizers import SGD,Adam,RMSprop
from keras.layers import *
import os
from keras import backend as K
from keras.engine.topology import Layer
import h5py
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pandas as pd
import numpy as np
import keras

from gensim.models import Word2Vec

Using TensorFlow backend.


In [2]:
#coding=utf-8
'''
Single model may achieve LB scores at around 0.043
Don't need to be an expert of feature engineering
All you need is a GPU!!!!!!!

The code is tested on Keras 2.0.0 using Theano backend, and Python 3.5

referrence Code:https://www.kaggle.com/lystdo/lstm-with-word2vec-embeddings
'''

########################################
## import packages
########################################
import os
import re
import csv
import codecs
import numpy as np
import pandas as pd

########################################
## set directories and parameters
########################################



from keras import backend as K
from keras.engine.topology import Layer
# from keras import initializations
from keras import initializers, regularizers, constraints

np.random.seed(2020)

class Attention(Layer):
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:

        """
        self.supports_masking = True
        # self.init = initializations.get('glorot_uniform')
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias

        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.step_dim = input_shape[1]
        assert len(input_shape) == 3 # batch ,timestep , num_features
        print(input_shape)
        self.W = self.add_weight((input_shape[-1],), #num_features
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),#timesteps
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim
        print(K.reshape(x, (-1, features_dim)))# n, d
        print(K.reshape(self.W, (features_dim, 1)))# w= dx1
        print(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))))#nx1

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))#batch,step
        print(eij)
        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        print(a)
        a = K.expand_dims(a)
        print("expand_dims:")
        print(a)
        print("x:")
        print(x)
        weighted_input = x * a
        print(weighted_input.shape)
#         return weighted_input
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
#         return input_shape[0], input_shape[-1]
#         return input_shape[0],self.step_dim, self.features_dim
        return input_shape[0], self.features_dim


In [12]:
# Position_Embedding
#! -*- coding: utf-8 -*-
#%%

class Position_Embedding(Layer):
 
    def __init__(self, size=None, mode='sum', **kwargs):
        self.size = size #必须为偶数
        self.mode = mode
        super(Position_Embedding, self).__init__(**kwargs)
 
    def call(self, x):
        if (self.size == None) or (self.mode == 'sum'):
            self.size = int(x.shape[-1])
        batch_size,seq_len = K.shape(x)[0],K.shape(x)[1]
        position_j = 1. / K.pow(10000., \
                                 2 * K.arange(self.size / 2, dtype='float32' \
                               ) / self.size)
        position_j = K.expand_dims(position_j, 0)
        position_i = K.cumsum(K.ones_like(x[:,:,0]), 1)-1 #K.arange不支持变长，只好用这种方法生成
        position_i = K.expand_dims(position_i, 2)
        position_ij = K.dot(position_i, position_j)
        position_ij = K.concatenate([K.cos(position_ij), K.sin(position_ij)], 2)
        if self.mode == 'sum':
            return position_ij + x
        elif self.mode == 'concat':
            return K.concatenate([position_ij, x], 2)
 
    def compute_output_shape(self, input_shape):
        if self.mode == 'sum':
            return input_shape
        elif self.mode == 'concat':
            return (input_shape[0], input_shape[1], input_shape[2]+self.size)
        
# attention

class Attention(Layer):
 
    def __init__(self, nb_head=8, size_per_head=32, **kwargs):
        self.nb_head = nb_head
        self.size_per_head = size_per_head
        self.output_dim = nb_head*size_per_head
        super(Attention, self).__init__(**kwargs)
 
    def build(self, input_shape):
        self.WQ = self.add_weight(name='WQ',
                                  shape=(input_shape[0][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WK = self.add_weight(name='WK',
                                  shape=(input_shape[1][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WV = self.add_weight(name='WV',
                                  shape=(input_shape[2][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        super(Attention, self).build(input_shape)
 
    def Mask(self, inputs, seq_len, mode='mul'):
        if seq_len == None:
            return inputs
        else:
            mask = K.one_hot(seq_len[:,0], K.shape(inputs)[1])
            mask = 1 - K.cumsum(mask, 1)
            for _ in range(len(inputs.shape)-2):
                mask = K.expand_dims(mask, 2)
            if mode == 'mul':
                return inputs * mask
            if mode == 'add':
                return inputs - (1 - mask) * 1e12
 
    def call(self, x):
        #如果只传入Q_seq,K_seq,V_seq，那么就不做Mask
        #如果同时传入Q_seq,K_seq,V_seq,Q_len,V_len，那么对多余部分做Mask
        if len(x) == 3:
            Q_seq,K_seq,V_seq = x
            Q_len,V_len = None,None
        elif len(x) == 5:
            Q_seq,K_seq,V_seq,Q_len,V_len = x
        #对Q、K、V做线性变换
        Q_seq = K.dot(Q_seq, self.WQ)
        Q_seq = K.reshape(Q_seq, (-1, K.shape(Q_seq)[1], self.nb_head, self.size_per_head))
        Q_seq = K.permute_dimensions(Q_seq, (0,2,1,3))
        K_seq = K.dot(K_seq, self.WK)
        K_seq = K.reshape(K_seq, (-1, K.shape(K_seq)[1], self.nb_head, self.size_per_head))
        K_seq = K.permute_dimensions(K_seq, (0,2,1,3))
        V_seq = K.dot(V_seq, self.WV)
        V_seq = K.reshape(V_seq, (-1, K.shape(V_seq)[1], self.nb_head, self.size_per_head))
        V_seq = K.permute_dimensions(V_seq, (0,2,1,3))
        #计算内积，然后mask，然后softmax
        A = K.batch_dot(Q_seq, K_seq, axes=[3,3]) / self.size_per_head**0.5
        A = K.permute_dimensions(A, (0,3,2,1))
        A = self.Mask(A, V_len, 'add')
        A = K.permute_dimensions(A, (0,3,2,1))
        A = K.softmax(A)
        #输出并mask
        O_seq = K.batch_dot(A, V_seq, axes=[3,2])
        O_seq = K.permute_dimensions(O_seq, (0,2,1,3))
        O_seq = K.reshape(O_seq, (-1, K.shape(O_seq)[1], self.output_dim))
        O_seq = self.Mask(O_seq, Q_len, 'mul')
        return O_seq
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], self.output_dim)

In [16]:
def buid_model():
    
    S_inputs = Input(shape=(200,),name='main_input',dtype='int32')
    mask = Input(shape=(200,),name='mask',dtype='int32')
    
    with h5py.File('../../get_w2v_feat/w2v1_pre_ad/size_32_w100_count_0/embeddings_matrix_ad_size_32_w100_count_0.h5','r') as f:
        embeddings_matrix = np.array(f.get('embeddings_matrix'))
    

    EMBEDDING_DIM = 32 #词向量维度
    MAX_SEQUENCE_LENGTH = 200
    embeddings = Embedding(input_dim = len(embeddings_matrix), # 字典长度
                                output_dim = EMBEDDING_DIM, # 词向量 长度（100）
                                weights=[embeddings_matrix], # 重点：预训练的词向量系数
                                input_length=MAX_SEQUENCE_LENGTH, # 每句话的 最大长度（必须padding） 
                                trainable=False, # 是否在 训练的过程中 更新词向量
                                mask_zero = False)(S_inputs)
    
    encodings = Position_Embedding(32)(embeddings)
    encodings = Add()([embeddings, encodings])
    x = Attention(8, 64)([encodings, encodings, encodings,mask,mask])#添加mask
    O_seq = GlobalAveragePooling1D()(x)
    
    # model1:multi_lstm_multi_attention_aver_ad_age_adm_w100_1.376-0.428-003-1.372-0.430.hdf5
#     embeddings = Bidirectional(LSTM(32,activation='softsign',return_sequences=True))(embeddings)
#     O_seq = Attention(8,32)([embeddings,embeddings,embeddings])
#     O_seq = GlobalAveragePooling1D()(O_seq)
    
    # model2:
#     embeddings = Bidirectional(LSTM(32,activation='softsign',return_sequences=True))(embeddings)
#     O_seq = Attention(8,64)([embeddings,embeddings,embeddings])
#     O_seq = GlobalMaxPool1D()(O_seq)
    
#     O_seq = Attention()(O_seq)
#     O_seq = GlobalMaxPool1D()(O_seq)
    outputs = Dense(10, activation='softmax', name='main_output')(O_seq)

    # 定义一个具有两个输入输出的模型
    model = keras.models.Model(inputs=[S_inputs,mask],#,auxiliary_input],
                               outputs=[outputs])  # 这里的输入输出顺序与fit时一致就好
    
#     model.layers[1].trainable = False
    
#     opt = RMSprop(lr=0.01,  clipnorm=1.0)
    opt = Adam(lr=0.01)
    model.compile(optimizer=opt,
                  sample_weight_mode='None',#"temporal",
                  loss={'main_output': 'categorical_crossentropy'},
                 metrics=['accuracy'])
    print(model.summary())
    return model

#     print(model.summary())


In [17]:
def data_load():
    print('loading data ... \n')

    with h5py.File('../../get_w2v_feat/w2v1_pre_ad/size_32_w100_count_0/word_train_ad_w2v.h5', 'r') as f:
        data = np.array(f.get('word_data'))

    label = pd.read_csv('../../train_preliminary/user.csv').sort_values(by=['user_id'])

    train_x, test_x, train_y, test_y = train_test_split(data, label, test_size=0.2, random_state=2020)

    train_y_age = train_y['age'].values - 1
    train_y_age = keras.utils.np_utils.to_categorical(train_y_age, num_classes=10)
    train_y_gender = train_y['gender'].values - 1

    test_y_age = test_y['age'].values - 1
    test_y_age = keras.utils.np_utils.to_categorical(test_y_age, num_classes=10)
    test_y_gender = test_y['gender'].values - 1
    
    with h5py.File('../../get_w2v_feat/w2v1_pre_ad/mask_train.h5','r') as f:
        mask = np.array(f.get('word_data'))
    train_mask, test_mask = train_test_split(mask, test_size=0.2, random_state=2020)
    
    print('get data ... \n')

    return train_x, test_x, train_y_age, train_y_gender,test_y_age,test_y_gender,train_mask, test_mask

In [18]:
model = buid_model()
print('lstm model geted...\n')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 200, 32)      96875552    main_input[0][0]                 
__________________________________________________________________________________________________
position__embedding_4 (Position (None, 200, 32)      0           embedding_5[0][0]                
__________________________________________________________________________________________________
add_4 (Add)                     (None, 200, 32)      0           embedding_5[0][0]                
                                                                 position__embedding_4[0][0]      
__________

In [19]:
train_x, test_x, train_y_age, train_y_gender,test_y_age,test_y_gender,train_mask, test_mask = data_load()

loading data ... 

get data ... 



In [ ]:
def get_filename_for_saving(save_dir):
    return os.path.join(save_dir,
                        "_multi_attention_aver_ad_age_adm_w100_{val_loss:.3f}-{val_acc:.3f}-{epoch:03d}-{loss:.3f}-{acc:.3f}.hdf5")

print('lstm model fit...\n')
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=get_filename_for_saving(''),
    save_best_only=False)
stopping = keras.callbacks.EarlyStopping(patience=8)
reduce_lr = keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=2, min_lr=0.0001)


model.fit({'main_input': train_x,'mask' : train_mask },#,'aux_input': train_x_sta},
          {'main_output': train_y_age},
          epochs=100,
          batch_size=256,
          validation_data=({'main_input': test_x,'mask' : test_mask},#,'aux_input': test_x_sta},
                           {'main_output': test_y_age}),
          callbacks=[checkpointer, reduce_lr, stopping])

lstm model fit...

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 720000 samples, validate on 180000 samples
Epoch 1/100
524800/720000 [====================>.........] - ETA: 32:08 - loss: 2.0852 - acc: 0.2249

In [ ]:
# 加载模型
from keras.utils import CustomObjectScope
with CustomObjectScope({'Attention': Attention}):
    lstm_model_path_age = 'multi_lstm_multi_attention_aver_ad_age_adm_w100_1.376-0.428-003-1.372-0.430.hdf5'
    model_age = keras.models.load_model(lstm_model_path_age)
    
model_age.layers[3].size_per_head